In [1]:
import random
import pandas as pd
import yaml
import joblib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from modeling import WithDropout
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

team_rb_stats = pd.read_csv("data/team_rushers.csv", index_col=0)
team_qb_stats = pd.read_csv("data/team_qb_stats.csv", index_col=0)
team_receiver_stats = pd.read_csv("data/team_receiver_stats.csv", index_col=0)
team_stats = pd.read_csv("data/agg/team_stats.csv", index_col=0).rename(
    {"recent_team": "team_name"}, axis=1
)
opp_stats = pd.read_csv("data/agg/opp_stats.csv", index_col=0).rename(
    {"opponent_team": "team_name"}, axis=1
)

In [2]:
players = pd.read_parquet("data/agg/player_weekly_agg.parquet").fillna(0)



In [3]:
players.loc[(players.player_id=='00-0036322') & (players.season==2024)].iloc[-3]
# will ship '00-0039746'

season                                                                           2024
team_name                                                                         MIN
week                                                                             11.0
game_type                                                                         REG
depth_team                                                                        1.0
last_name                                                                   Jefferson
first_name                                                                     Justin
football_name                                                                  Justin
formation                                                                     Offense
gsis_id                                                                    00-0036322
jersey_number                                                                    18.0
position                                              

In [4]:
model_path = "models/run_yards_gained.pt"
run_yards_model = WithDropout(n_in=22, n_hidden=1000, n_out=130)
run_yards_model.load_state_dict(torch.load(model_path, weights_only=True))

air_yards_path = "models/air_yards.pt"
air_yards_model = WithDropout(n_in=26, n_out=119)
air_yards_model.load_state_dict(torch.load(air_yards_path, weights_only=True))


yac_model_path = "models/yac.pt"
yac_model = WithDropout(n_in=27, n_out=125)
yac_model.load_state_dict(torch.load(yac_model_path, weights_only=True))


<All keys matched successfully>

In [5]:
stat_cols = [
    "completions",
    "attempts",
    "passing_yards",
    "passing_tds",
    "interceptions",
    "sacks",
    "sack_yards",
    "sack_fumbles",
    "sack_fumbles_lost",
    "passing_air_yards",
    "passing_yards_after_catch",
    "passing_first_downs",
    "passing_epa",
    "passing_2pt_conversions",
    "pacr",
    "dakota",
    "carries",
    "rushing_yards",
    "rushing_tds",
    "rushing_fumbles",
    "rushing_fumbles_lost",
    "rushing_first_downs",
    "rushing_epa",
    "rushing_2pt_conversions",
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "receiving_fumbles",
    "receiving_fumbles_lost",
    "receiving_air_yards",
    "receiving_yards_after_catch",
    "receiving_first_downs",
    "receiving_epa",
    "receiving_2pt_conversions",
    "racr",
    "target_share",
    "air_yards_share",
    "wopr",
    "special_teams_tds",
    "fantasy_points",
    "fantasy_points_ppr",
]

In [6]:
class Player:
    def __init__(self, d):
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.depth_team = int(d["dense_depth"])
        self.stats = {x: 0 for x in stat_cols}
        self.stats["air_yards"] = 0
        self.stats["yac"] = 0
        self.features = d.to_dict()

    def show_headshot(self):
        url = self.headshot_url
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful
        img = Image.open(BytesIO(response.content))
        plt.imshow(img)
        plt.axis("off")  # Turn off axis for cleaner display
        plt.show()

    def __getattr__(self, name):
        # Redirect attribute access to the stats dictionary
        if name in self.stats:
            return self.stats[name]
        elif name in self.features:
            return self.features[name]
        raise AttributeError(
            f"'{self.__class__.__name__}' object has no attribute '{name}'"
        )

    def __setattr__(self, name, value):
        if name in {
            "stats",
            "features",
            "name",
            "id",
            "depth_team",
        }:  # Handle direct attributes
            super().__setattr__(name, value)
        elif name in self.stats:  # Redirect updates to stats dictionary
            self.stats[name] = value
        elif name in self.features:  # Redirect updates to features dictionary
            self.features[name] = value
        else:
            raise AttributeError(f"Cannot set unknown attribute '{name}'")

    def stats_to_dict(self):
        out = {}
        out["name"] = self.name
        out["id"] = self.id
        out.update(self.stats)

        return out

In [26]:
def fetch_row_or_latest(df, team, season, week):
    df = df.loc[(df.team_name == team) & (df.season == season)]
    row = df.loc[(df.week == min(df.week.max(), week))].to_dict(orient="records")[0]
    return row


class QB(Player):
    def __init__(self, d):  # noqa: F811
        super().__init__(d)
        self.name = d["full_name"]
        self.features = d.to_dict()

    def __repr__(self):
        return f"QB:{self.name} has {self.completions} completions for {self.passing_yards} yards"


class RB(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.features = d.to_dict()

    def __repr__(self):
        return (
            f"RB:{self.name} has {self.carries} carries for {self.rushing_yards} yards"
        )


class WR(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]

    def __repr__(self):
        return f"WR:{self.name} has {self.receptions} receptions for {self.receiving_yards} yards"


class TE(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]

    def __repr__(self):
        return f"TE:{self.name} has {self.receptions} receptions for {self.receiving_yards} yards"


class Team:
    def __init__(self, name: str, season: int, week: int, use_current_injuries=False):
        self.name = name
        self.score = 0
        self.plays = 0
        self.features = {"last_rusher_drive": -1, "last_rusher_team": -1}
        self.team_stats = fetch_row_or_latest(team_stats, self.name, season, week)
        self.opp_stats = fetch_row_or_latest(opp_stats, self.name, season, week)
        self.roster = players.loc[
            (players.team_name == name) & (players.season == season)
        ]
        self.roster = self.roster.loc[
            (self.roster.week == min(self.roster.week.max(), week))
            & (self.roster.formation == "Offense")
            & (self.roster.position.isin(["QB", "WR", "TE", "RB"]))
        ].sort_values(by="dense_depth")

        self.QBs = self.get_players_by_position("QB")
        self.RBs = self.get_players_by_position("RB")
        self.WRs = self.get_players_by_position("WR")
        self.TEs = self.get_players_by_position("TE")
        self.players = self.QBs + self.RBs + self.WRs + self.TEs
        self.rb_stats = fetch_row_or_latest(team_rb_stats, self.name, season, week)

        self.team_receiver_stats = fetch_row_or_latest(
            team_receiver_stats, self.name, season, week
        )

    def get_players_by_position(self, position: str):
        """Filter players by position and create player objects."""
        with pd.option_context("future.no_silent_downcasting", True):
            position_data = self.roster[(self.roster["position"] == position)].fillna(0)
        # Create player objects based on position
        players = []
        for _, player_data in position_data.iterrows():
            if position == "WR":
                players.append(WR(player_data))
            elif position == "RB":
                players.append(RB(player_data))
            elif position == "QB":
                players.append(QB(player_data))
            elif position == "TE":
                players.append(TE(player_data))
        return players

    def get_depth_pos(self, pos: str, depth: int):
        """input a position and team depth, to get the player
        used to go from ML output -> player object"""
        if pos == "WR":
            for player in self.WRs:
                if player.depth_team == depth:
                    return player
        if pos == "RB":
            for player in self.RBs:
                if player.depth_team == depth:
                    return player
        if pos == "TE":
            for player in self.TEs:
                if player.depth_team == depth:
                    return player
        if pos == "QB":
            for player in self.QBs:
                if player.depth_team == depth:
                    return player
        raise ValueError("You want a player that does not exist")

    def game_results(self):
        return [{'team_name':self.name} | x.stats_to_dict() for x in self.players]

    def __repr__(self):
        return f"{self.name} has {self.score} points"


In [27]:
vik = Team("MIN", 2024, 13)
det = Team("DET", 2024, 13)
vik.game_results()

[{'team_name': 'MIN',
  'name': 'Sam Darnold',
  'id': '00-0034869',
  'completions': 0,
  'attempts': 0,
  'passing_yards': 0,
  'passing_tds': 0,
  'interceptions': 0,
  'sacks': 0,
  'sack_yards': 0,
  'sack_fumbles': 0,
  'sack_fumbles_lost': 0,
  'passing_air_yards': 0,
  'passing_yards_after_catch': 0,
  'passing_first_downs': 0,
  'passing_epa': 0,
  'passing_2pt_conversions': 0,
  'pacr': 0,
  'dakota': 0,
  'carries': 0,
  'rushing_yards': 0,
  'rushing_tds': 0,
  'rushing_fumbles': 0,
  'rushing_fumbles_lost': 0,
  'rushing_first_downs': 0,
  'rushing_epa': 0,
  'rushing_2pt_conversions': 0,
  'receptions': 0,
  'targets': 0,
  'receiving_yards': 0,
  'receiving_tds': 0,
  'receiving_fumbles': 0,
  'receiving_fumbles_lost': 0,
  'receiving_air_yards': 0,
  'receiving_yards_after_catch': 0,
  'receiving_first_downs': 0,
  'receiving_epa': 0,
  'receiving_2pt_conversions': 0,
  'racr': 0,
  'target_share': 0,
  'air_yards_share': 0,
  'wopr': 0,
  'special_teams_tds': 0,
  'fan

In [28]:
class GameState:
    def __init__(self, home, away, config, **kwargs):
        self.home = home
        self.away = away
        self.quarter = 1
        self.possession = None
        self.defending = None
        self.down = 1
        self.ydstogo = 10
        self.ball_position = 65  # Yardline (0-100), 0 is score, 100 is safety
        self.clock = 900  # Seconds in the current quarter (15 mins = 900 seconds)
        self.drive = 0
        self.home_timeouts = 3
        self.away_timeouts = 3
        self.pbp = []
        home.spread_line= kwargs.get('spread_line', -3)
        away.spread_line= -1 * self.home.spread_line
        self.total_line= kwargs.get('total_line', 42)
        self.player = None
        self.run_or_pass = joblib.load("models/run_or_pass.joblib")
        self.run_or_pass_cols = config["run_or_pass_cols"]
        self.choose_rusher = joblib.load("models/choose_rusher.joblib")
        self.choose_rusher_cols = config["choose_rusher_cols"]
        self.choose_receiver = joblib.load("models/choose_receiver.joblib")
        self.choose_receiver_cols = config["choose_receiver_cols"]
        self.air_yards_cols = config["air_yards_cols"]
        self.receiver_idx_to_pos = config["receiver_idx_to_pos"]
        self.rusher_idx_to_pos = config["rusher_idx_to_pos"]
        self.rush_yard_cols = config["rush_yard_cols"]
        self.complete_pass_cols = config["complete_pass_cols"]
        self.complete_pass_model = joblib.load("models/complete_pass.joblib")
        self.play_encoding = config["play_encoding"]
        self.wind = kwargs.get('wind', random.randint(0, 10))
        self.temp = kwargs.get('temp', random.randint(40, 90))
        self.play_functions = {
            "field_goal": self.field_goal,
            "no_play": self.run_play,
            "pass": self.pass_play,
            "punt": self.punt,
            "qb_kneel": self.qb_kneel,
            "qb_spike": self.qb_spike,
            "run": self.run_play,
        }

    def switch_poss(self):
        self.possession.features["last_rusher_drive"] = -1
        self.possession = self.away if self.possession == self.home else self.home
        self.defending = self.away if self.defending == self.home else self.home
        self.ball_position = 100 - min(self.ball_position, 99)
        self.down = 1
        self.ydstogo = min(10, self.ball_position)
        self.drive += 1
        return

    def kickoff(self):
        self.switch_poss()
        self.ball_position = 65
        pass

    def start_game(self):
        lost_kickoff = random.choice((self.home, self.away))
        self.possession = lost_kickoff
        self.kickoff()
        print(f"{self.possession.name} has won the kickoff")
        self.log_play("kickoff", 0)

    def collect_features(self, *argv):
        features = {}
        for arg in argv:
            features.update(arg)
        return features

    def play(self, team):
        team.plays += 1
        raw_features = self.collect_features(
            self.pbp[-1], team.team_stats, self.defending.opp_stats#, team.spread_line
        )
        features = [raw_features[key] for key in self.run_or_pass_cols]
        preds = self.run_or_pass.predict_proba([features])
        play_type_int = np.random.choice(len(preds[0]), p=preds[0])
        play_type = self.play_encoding.get(play_type_int, 1)
        yds = self.play_functions[play_type](team)
        self.log_play(play_type, yds)
        self.ydstogo -= yds
        self.ball_position -= yds
        self.clock -= random.randint(15, 40)
        self.td_check(team)
        self.check_downs(team)

    def log_play(self, play_type, yds, verbose=False):
        """Logs the context of the game state at each play."""
        play_data = {
            "possession": self.possession.name,
            "quarter": self.quarter,
            "down": self.down,
            "ydstogo": self.ydstogo,
            "goal_to_go": int(self.ball_position < 10),
            "yardline_100": self.ball_position,
            "total_home_score": self.home.score,
            "total_away_score": self.away.score,
            "posteam_score": self.possession.score,
            "defteam_score": self.home.score
            if self.possession == self.away
            else self.away.score,
            "score_differential": (self.home.score - self.away.score),
            "wind": self.wind,
            "temp": self.temp,
            "quarter_seconds_remaining": self.clock,
            "half_seconds_remaining": self.clock + (900 * (self.quarter % 2)),
            "game_seconds_remaining": self.clock + (900 * (4 - self.quarter)),
            "drive": self.drive,
            "spread_line": self.possession.spread_line,
            "total_line": self.total_line,
            "play_type": play_type,
            "yards_gained": yds,
            "player": self.player,
        }
        if verbose:
            print(
                f'{self.possession.name} {play_type} for {yds} yards, {self.pbp[-1]['yardline_100']} yd line,'
                + f' {self.pbp[-1]['ydstogo']} yds to go on {self.pbp[-1]['down']} down.'
                + f' {self.pbp[-1]['quarter_seconds_remaining'] // 60}:{self.pbp[-1]['quarter_seconds_remaining']  % 60} left'
            )
        self.pbp.append(play_data)

    def sample_run_yards(self, model, player):
        # rush_yards_cols = self.rush_yards_cols
        raw_features = self.collect_features(
            dict(self.pbp[-1]),
            player.features,
            self.possession.team_stats,
            self.defending.opp_stats,
        )
        x = [raw_features[key] for key in self.rush_yard_cols]
        x = torch.tensor(x)
        with torch.no_grad():
            preds = model(x.reshape(1, -1))[0]
            preds = torch.softmax(preds, 0)
        sample = (torch.multinomial(preds, 1)).item() - 30
        return min((sample, raw_features["yardline_100"]))

    def sample_air_and_yac(self, air_model, yac_model, player):
        raw_features = self.collect_features(
            dict(self.pbp[-1]),
            player.features,
            self.possession.team_stats,
            self.defending.opp_stats,
        )
        x = [raw_features[key] for key in self.air_yards_cols]
        x = torch.tensor(x)
        with torch.no_grad():
            preds = air_model(x.reshape(1, -1))[0]
            preds = torch.softmax(preds, 0)
        air_yards = (torch.multinomial(preds, 1)).item() - 20
        if air_yards >= self.ball_position:  # touchdown at catch
            return air_yards, 0

        x = torch.cat((x, torch.tensor([air_yards])))
        with torch.no_grad():
            preds = yac_model(x.reshape(1, -1))[0]
            preds = torch.softmax(preds, 0)
        yac = (torch.multinomial(preds, 1)).item() - 25
        return air_yards, min(yac, (self.ball_position - air_yards))

    def sample_completion(self, qb, receiver, air_yards):
        raw_features = self.collect_features(
            dict(self.pbp[-1]),
            receiver.features,
            self.possession.team_stats,
            self.defending.opp_stats,
        )
        raw_features["air_yards"] = air_yards
        qb_features = {(key + "_qb"): value for key, value in qb.features.items()}
        raw_features.update(qb_features)
        features = [raw_features[key] for key in self.complete_pass_cols]
        preds = self.complete_pass_model.predict_proba([features])
        receiver = np.random.choice(len(preds[0]), p=preds[0])
        return np.random.choice(len(preds[0]), p=preds[0])

    def run_play(self, team):
        raw_features = dict(self.pbp[-1])
        raw_features.update(team.rb_stats)
        raw_features.update(team.features)
        raw_features.update(self.defending.opp_stats)
        features = [raw_features[key] for key in self.choose_rusher_cols]
        preds = self.choose_rusher.predict_proba([features])
        rusher_idx = np.random.choice(len(preds[0]), p=preds[0])
        pos, depth = self.rusher_idx_to_pos[rusher_idx].split("_")
        try:
            player = team.get_depth_pos(pos, int(depth))
        except ValueError:
            print(f'{pos}{depth} for {team.name} does not exist')
            player = team.RBs[0]
        player.carries += 1
        yds = self.sample_run_yards(run_yards_model, player)
        player.rushing_yards += yds
        self.player = player.name
        team.features["last_rusher_team"] = rusher_idx
        team.features["last_rusher_drive"] = rusher_idx
        return yds

    def pass_play(self, team):
        passer = team.QBs[0]
        raw_features = dict(self.pbp[-1])
        raw_features.update(team.team_receiver_stats)
        raw_features.update(team.features)
        features = [raw_features[key] for key in self.choose_receiver_cols]
        preds = self.choose_receiver.predict_proba([features])
        receiver = np.random.choice(len(preds[0]), p=preds[0])
        pos, depth = self.receiver_idx_to_pos[receiver].split("_")
        receiver = team.get_depth_pos(pos, int(depth))
        passer.attempts += 1
        receiver.targets += 1
        air_yards, yac = self.sample_air_and_yac(air_yards_model, yac_model, receiver)
        qb_features = {}
        qb_features.update(passer.features)

        if self.sample_completion(passer, receiver, air_yards):
            passer.completions += 1
            receiver.receptions += 1
            yds = air_yards + yac
            receiver.air_yards += air_yards
            receiver.yac += yac
            receiver.receiving_yards += yds
            passer.passing_yards += yds
        else:
            yds = 0
        self.player = receiver.name
        return yds

    def punt(self, team):
        self.switch_poss()
        self.ball_position += random.randint(30, 60)
        if self.ball_position >= 100:
            self.ball_position = 20
        self.player = None
        return 0

    def field_goal(self, team):
        result = random.randint(0, 100)
        if result > (2 * self.ball_position):
            team.score += 3
            self.switch_poss()
            self.ball_position = 65
            # print(f'{team.name} scored a FG')
        else:
            # print(f'{team.name} missed FG')
            self.switch_poss()
        self.player = None
        return 0

    def qb_kneel(self, team):
        # Implementation of qb kneel play
        # print("QB kneel executed.")
        return -1

    def qb_spike(self, team):
        # Implementation of qb spike play
        # print("QB spike executed.")
        return 0

    def td_check(self, team):
        if self.ball_position <= 0:
            team.score += 7
            self.kickoff()
            # print(f'{team.name} scored a TD')
        return

    def check_downs(self, team):
        if self.ydstogo <= 0:
            self.ydstogo = 10
            self.down = 1
        elif self.down == 4:
            self.switch_poss()
        else:
            self.down += 1

    def play_quarter(self):
        self.clock = 900
        while self.clock > 0:
            self.play(self.possession)
        self.quarter += 1
        #print(f"{self.home.name}:{self.home.score}")
        #print(f"{self.away.name}:{self.away.score}")

    def play_game(self):
        while self.quarter <= 4:
            self.play_quarter()
            #print(self.quarter)

In [33]:
with open("models/feature_config.yaml", "r") as file:
    config = yaml.safe_load(file)

vik = Team("PHI", 2024, 11)
det = Team("KC", 2024, 11)
game = GameState(vik, det, config, **{'wind':25, 'temp':90})
game.start_game()
game.play_game()

PHI has won the kickoff


In [34]:
vik.QBs[0].stats

{'completions': 19,
 'attempts': 33,
 'passing_yards': 265,
 'passing_tds': 0,
 'interceptions': 0,
 'sacks': 0,
 'sack_yards': 0,
 'sack_fumbles': 0,
 'sack_fumbles_lost': 0,
 'passing_air_yards': 0,
 'passing_yards_after_catch': 0,
 'passing_first_downs': 0,
 'passing_epa': 0,
 'passing_2pt_conversions': 0,
 'pacr': 0,
 'dakota': 0,
 'carries': 8,
 'rushing_yards': 16,
 'rushing_tds': 0,
 'rushing_fumbles': 0,
 'rushing_fumbles_lost': 0,
 'rushing_first_downs': 0,
 'rushing_epa': 0,
 'rushing_2pt_conversions': 0,
 'receptions': 0,
 'targets': 0,
 'receiving_yards': 0,
 'receiving_tds': 0,
 'receiving_fumbles': 0,
 'receiving_fumbles_lost': 0,
 'receiving_air_yards': 0,
 'receiving_yards_after_catch': 0,
 'receiving_first_downs': 0,
 'receiving_epa': 0,
 'receiving_2pt_conversions': 0,
 'racr': 0,
 'target_share': 0,
 'air_yards_share': 0,
 'wopr': 0,
 'special_teams_tds': 0,
 'fantasy_points': 0,
 'fantasy_points_ppr': 0,
 'air_yards': 0,
 'yac': 0}

In [14]:
def run_sim(team1, team2,num_sims, config=config):
    results = []
    for i in range(num_sims):
        vik = Team(team1, 2024, 12)
        det = Team(team2, 2024, 12)
        game = GameState(vik, det, config)
        game.start_game()
        game.play_game()
        res = vik.game_results() + det.game_results()
        for result in res:
            result["game_number"] = i
        results += res
    return results
results = run_sim('PHI','LA',10)

PHI has won the kickoff
PHI has won the kickoff
LA has won the kickoff
LA has won the kickoff
PHI has won the kickoff
PHI has won the kickoff
PHI has won the kickoff
LA has won the kickoff
PHI has won the kickoff
LA has won the kickoff


In [16]:
df = pd.json_normalize(results)
df.groupby(["name", "id"])[
    [
        "receiving_yards",
        "targets",
        "receptions",
        "air_yards",
        "completions",
        "attempts",
        "rushing_yards",
        "carries",
    ]
].agg(['median','mean']).sort_values(by=[('rushing_yards','mean')],ascending=False)


receiving_yards        targets       receptions  \
                                       median   mean  median  mean     median   
name               id                                                           
Saquon Barkley     00-0034844            43.5   44.0     4.0   4.0        4.0   
Kyren Williams     00-0037840             0.0    4.8     0.0   0.7        0.0   
Kenneth Gainwell   00-0036919             0.0    2.1     0.0   0.2        0.0   
Blake Corum        00-0039738             0.0   -0.2     0.0   0.1        0.0   
Ronnie Rivers      00-0037557             0.0    0.0     0.0   0.0        0.0   
A.J. Brown         00-0035676            66.0   85.5     7.0   7.3        4.5   
Will Shipley       00-0039746             0.0    0.0     0.0   0.0        0.0   
Tyler Johnson      00-0036427             0.0    0.0     0.0   0.0        0.0   
Tutu Atwell        00-0036849             8.5    8.7     1.5   2.1        1.0   
Tanner McKee       00-0038400             0.0    0.0     0.0   0.0        0.0   
Stetson Bennett    00-0039107             0.0    0.0     0.0   0.0        0.0   
Puka Nacua         00-0039075            51.5   65.1     7.0   6.8        4.5   
Matthew Stafford   00-0026498             0.0    0.0     0.0   0.0        0.0   
Kenny Pickett      00-0038102             0.0    0.0     0.0   0.0        0.0   
Jordan Whittington 00-0039751             4.5   11.9     1.0   1.2        0.5   
Johnny Wilson      00-0039236             0.0    0.0     0.0   0.0        0.0   
Jimmy Garoppolo    00-0031345             0.0    0.0     0.0   0.0        0.0   
Jalen Hurts        00-0036389             0.0    0.0     0.0   0.0        0.0   
Jahan Dotson       00-0037741             0.0    0.9     1.0   1.1        0.0   
Hunter Long        00-0037004             0.0    0.0     0.0   0.0        0.0   
Grant Calcaterra   00-0037086             0.0    5.4     1.5   1.8        0.5   
Demarcus Robinson  00-0032775            12.0   14.5     2.0   2.5        1.0   
DeVonta Smith      00-0036912            45.0   45.3     4.5   5.2        4.0   
Davis Allen        00-0039074             0.0    3.0     0.0   0.6        0.0   
Dallas Goedert     00-0034351            27.5   32.8     3.0   4.3        3.0   
Cooper Kupp        00-0033908           156.5  150.0    12.5  12.1        8.5   
Colby Parkinson    00-0036244             1.0    4.8     1.0   0.9        0.5   
Xavier Smith       00-0038359             0.0    0.0     0.0   0.0        0.0   

                                   air_yards        completions        \
                              mean    median   mean      median  mean   
name               id                                                   
Saquon Barkley     00-0034844  3.6       9.0   12.7         0.0   0.0   
Kyren Williams     00-0037840  0.5       0.0    0.8         0.0   0.0   
Kenneth Gainwell   00-0036919  0.2       0.0   -0.3         0.0   0.0   
Blake Corum        00-0039738  0.1       0.0   -0.4         0.0   0.0   
Ronnie Rivers      00-0037557  0.0       0.0    0.0         0.0   0.0   
A.J. Brown         00-0035676  5.4      55.0   62.2         0.0   0.0   
Will Shipley       00-0039746  0.0       0.0    0.0         0.0   0.0   
Tyler Johnson      00-0036427  0.0       0.0    0.0         0.0   0.0   
Tutu Atwell        00-0036849  1.4       4.0    3.4         0.0   0.0   
Tanner McKee       00-0038400  0.0       0.0    0.0         0.0   0.0   
Stetson Bennett    00-0039107  0.0       0.0    0.0         0.0   0.0   
Puka Nacua         00-0039075  4.7      33.5   35.5         0.0   0.0   
Matthew Stafford   00-0026498  0.0       0.0    0.0        18.0  19.1   
Kenny Pickett      00-0038102  0.0       0.0    0.0         0.0   0.0   
Jordan Whittington 00-0039751  1.1       0.0    1.0         0.0   0.0   
Johnny Wilson      00-0039236  0.0       0.0    0.0         0.0   0.0   
Jimmy Garoppolo    00-0031345  0.0       0.0    0.0         0.0   0.0   
Jalen Hurts        00-0036389  0.0       0.0    0.0       

In [ ]:
sched = pd.read_csv('week_13.csv',index_col=0)
sched

,game_id,season,week,home_team,away_team,home_moneyline,spread_line,total_line,temp,wind
6885,2024_13_CHI_DET,2024,13,DET,CHI,-440.0,9.5,47.5,50.0,8.0
6886,2024_13_NYG_DAL,2024,13,DAL,NYG,-170.0,3.5,37.5,50.0,8.0
6887,2024_13_MIA_GB,2024,13,GB,MIA,-166.0,3.5,47.5,50.0,8.0
6888,2024_13_LV_KC,2024,13,KC,LV,-700.0,13.0,42.5,50.0,8.0
6889,2024_13_LAC_ATL,2024,13,ATL,LAC,105.0,-1.0,47.5,50.0,8.0
6890,2024_13_PIT_CIN,2024,13,CIN,PIT,-162.0,3.0,47.5,50.0,8.0
6891,2024_13_HOU_JAX,2024,13,JAX,HOU,154.0,-4.0,43.5,50.0,8.0
6892,2024_13_ARI_MIN,2024,13,MIN,ARI,-180.0,3.5,45.0,50.0,8.0
6893,2024_13_IND_NE,2024,13,NE,IND,120.0,-2.5,42.5,50.0,8.0
6894,2024_13_SEA_NYJ,2024,13,NYJ,SEA,114.0,-2.0,42.5,50.0,8.0


In [ ]:
game1 = sched.iloc[0].to_dict()
game1

{'game_id': '2024_13_CHI_DET',
 'season': 2024,
 'week': 13,
 'home_team': 'DET',
 'away_team': 'CHI',
 'home_moneyline': -440.0,
 'spread_line': 9.5,
 'total_line': 47.5,
 'temp': 50.0,
 'wind': 8.0}

In [ ]:
results = []
for t in range(1):
#for t in range(len(sched)):
    game1 = sched.iloc[t].to_dict()
    for i in range(5):
        try:
            vik = Team(game1['home_team'], game1['season'], game1['week'])
            det = Team(game1['away_team'], game1['season'], game1['week'])
            game = GameState(vik, det, config,**game1 )
            game.start_game()
            game.play_game()
            res = vik.game_results() + det.game_results()
            for player in res:
                player["game_number"] = i
            results += res
        except:
            print('oops')

DET has won the kickoff
CHI has won the kickoff
CHI has won the kickoff
DET has won the kickoff
DET has won the kickoff
DET has won the kickoff
CHI has won the kickoff
DET has won the kickoff
CHI has won the kickoff
DET has won the kickoff
DET has won the kickoff
CHI has won the kickoff
DET has won the kickoff
CHI has won the kickoff
CHI has won the kickoff
DET has won the kickoff
DET has won the kickoff
oops
DET has won the kickoff
DET has won the kickoff
CHI has won the kickoff
CHI has won the kickoff
CHI has won the kickoff
DET has won the kickoff
DET has won the kickoff
CHI has won the kickoff
CHI has won the kickoff
DET has won the kickoff
CHI has won the kickoff
CHI has won the kickoff
DET has won the kickoff
CHI has won the kickoff
CHI has won the kickoff
DET has won the kickoff
CHI has won the kickoff
CHI has won the kickoff
oops
CHI has won the kickoff
DET has won the kickoff
DET has won the kickoff
CHI has won the kickoff
CHI has won the kickoff
CHI has won the kickoff
DET ha

In [ ]:
weekly = pd.read_csv('data/weekly.csv',index_col=0)
pit = weekly.loc[
    (weekly.week==12)
    & (weekly.season==2024)
    & ((weekly.recent_team=='PIT') | (weekly.recent_team=='CLE'))
    ]

In [ ]:
df = pd.json_normalize(results)
check_stats =     [
        "receiving_yards",
        "targets",
        "receptions",
        "completions",
        "attempts",
        "rushing_yards",
        "carries",
    ]

preds = df.groupby(["name", "team_name","id"])[
    [
        "receiving_yards",
        "targets",
        "receptions",
        "air_yards",
        "completions",
        "attempts",
        "rushing_yards",
        "carries",
    ]
].quantile(0.5).sort_values(by=['receiving_yards'],ascending=False).reset_index()
preds

,name,team_name,id,receiving_yards,targets,receptions,air_yards,completions,attempts,rushing_yards,carries
0,Jauan Jennings,SF,00-0036259,130.0,13.0,10.0,87.0,0.0,0.0,0.0,0.0
1,Brock Bowers,LV,00-0039338,125.0,15.0,11.0,46.5,0.0,0.0,0.0,0.0
2,Jaxon Smith-Njigba,SEA,00-0038543,99.5,9.0,7.0,71.5,0.0,0.0,0.0,0.0
3,Amon-Ra St. Brown,DET,00-0036963,99.0,8.0,7.0,63.5,0.0,0.0,0.0,0.0
4,Cooper Kupp,LA,00-0033908,89.0,9.0,6.0,63.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
441,Jake Haener,NO,00-0038998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
442,Jake Browning,CIN,00-0035100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
443,Jake Bobo,SEA,00-0038752,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
444,Jaheim Bell,NE,00-0039420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pp = pd.read_csv("C:/Github/NBA_betting_model/Lines/current/pp_current.csv",index_col=0)
pp.loc[pp.league_name=='NFL']['stat'].unique()

array(['Rush+Rec TDs', 'Receiving Yards', 'Receptions', 'Fantasy Score',
       'Rec Targets', 'Longest Reception',
       'Receiving Yards in First 2 Receptions', 'Pass TDs',
       'Pass Completions', 'Completion Percentage', 'Pass Attempts',
       'Pass+Rush Yds', 'Pass Yards', 'Rush Yards', 'INT',
       'Rush Yards in First 5 Attempts', 'Rush Attempts', 'Rush+Rec Yds',
       'Longest Rush', 'FG', 'Pass Yards (Combo)', 'Rush Yards (Combo)',
       'Receiving Yards (Combo)', 'Rush+Rec Yds (Combo)', 'Sacks',
       'Longest FG Made Yds (Combo)', 'Shortest FG Made Yds (Combo)',
       'Kicking Points', 'FG Made (Combo)', 'Punts Inside 20'],
      dtype=object)

In [ ]:
preds2pp = {
    'receiving_yards': 'Receiving Yards',
    'receptions': 'Receptions',
    'targets': 'Rec Targets',
    'completions': 'Pass Completions',
    'attempts': 'Pass Attempts',
    'passing_yards': 'Pass Yards',
    'carries': 'Rush Attempts',
}

In [ ]:
df.loc[df.name=='Brock Bowers', 'receiving_yards']

4025     64
4052    175
4079    159
4106    187
4133    121
4160    126
4187     38
4214     55
4241    167
4268    199
4295    179
4322    172
4349    148
4376     81
4403    206
4430    123
4457    167
4484     76
4511    100
4538    101
4565    126
4592    161
4619     55
4646    259
4673    145
4700    220
4727     63
4754     75
4781    146
4808     28
4835    196
4862    208
4889    124
4916     93
4943    112
4970     92
4997     93
5024     90
5051    151
5078    168
5105     86
5132    108
5159    128
5186    123
5213    179
5240    100
5267    189
5294    186
5321     54
5348     47
Name: receiving_yards, dtype: int64

In [ ]:
melted=preds.melt(['name','team_name','id'])
melted['stat'] = melted['variable'].apply(lambda x: preds2pp.get(x,x))
comb = melted.merge(pp.loc[(pp.league_id==9)
                           & (pp.alt_line=='standard')],left_on=['name','team_name','stat'], right_on=['player','team','stat'])
comb['Z'] = (comb['value'] - comb['line'] )/ (comb['value'])
comb.loc[comb.Z > -50].sort_values(by='Z')

,name,team_name,id,variable,value,stat,player,team,line,alt_line,opp,league_id,league_name,event_time,pp_player_id,date,scrape_time,prop_id,event_id,Z
70,Brock Wright,DET,00-0036754,receiving_yards,1.0,Receiving Yards,Brock Wright,DET,4.5,standard,CHI,9,NFL,2024-11-28 11:00:00-06:00,215927,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,fd12a231-e,a43ef946-7,-3.500000
67,Jalen Tolbert,DAL,00-0037666,receiving_yards,6.0,Receiving Yards,Jalen Tolbert,DAL,21.5,standard,NYG,9,NFL,2024-11-28 15:00:00-06:00,211429,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,e196374e-e,00cbc165-f,-2.583333
68,Rico Dowdle,DAL,00-0036139,receiving_yards,4.5,Receiving Yards,Rico Dowdle,DAL,15.5,standard,NYG,9,NFL,2024-11-28 15:00:00-06:00,211732,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,c57c0b13-b,00cbc165-f,-2.444444
55,Mike Evans,TB,00-0031408,receiving_yards,18.0,Receiving Yards,Mike Evans,TB,59.5,standard,CAR,9,NFL,2024-12-01 15:00:00-06:00,211278,2024-12-01 00:00:00-06:00,2024-11-27 19:40:00-06:00,42be8414-e,77c97f1c-c,-2.305556
66,Jahmyr Gibbs,DET,00-0039139,receiving_yards,6.5,Receiving Yards,Jahmyr Gibbs,DET,20.5,standard,CHI,9,NFL,2024-11-28 11:00:00-06:00,206307,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,46c0b425-e,a43ef946-7,-2.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Malik Washington,MIA,00-0039880,receiving_yards,10.5,Receiving Yards,Malik Washington,MIA,5.5,standard,GB,9,NFL,2024-11-28 19:00:00-06:00,216934,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,72ddd318-9,73168dea-1,0.476190
43,Josh Jacobs,GB,00-0035700,receiving_yards,30.0,Receiving Yards,Josh Jacobs,GB,15.5,standard,MIA,9,NFL,2024-11-28 19:00:00-06:00,206303,2024-11-28 00:00:00-06:00,2024-11-27 19:40:00-06:00,a64149b4-d,73168dea-1,0.483333
45,Josh Oliver,MIN,00-0035249,receiving_yards,29.0,Receiving Yards,Josh Oliver,MIN,14.5,standard,ARI,9,NFL,2024-12-01 12:00:00-06:00,212257,2024-12-01 00:00:00-06:00,2024-11-27 19:40:00-06:00,711af35e-a,22622b6e-5,0.500000
0,Brock Bowers,LV,00-0039338,receiving_yards,125.0,Receiving Yards,Brock Bowers,LV,61.5,standard,KC,9,NFL,2024-11-29 14:00:00-06:00,210636,2024-11-29 00:00:00-06:00,2024-11-27 19:40:00-06:00,3e740a34-1,278e4fb4-3,0.508000


In [ ]:
comb.groupby(['stat'])[['value','line']].mean()

,value,line
stat,,
Pass Attempts,30.055556,31.666667
Pass Completions,19.555556,20.944444
Rec Targets,6.448718,6.179487
Receiving Yards,39.506579,39.947368
Receptions,3.690476,3.952381
Rush Attempts,13.250000,11.000000


In [ ]:
comb = preds.merge(pit[['player_id'] + check_stats], left_on='id', right_on='player_id', suffixes=['_pred','_act']).drop('player_id',axis=1)
comb

,name,id,receiving_yards_pred,targets_pred,receptions_pred,air_yards,completions_pred,attempts_pred,rushing_yards_pred,carries_pred,receiving_yards_act,targets_act,receptions_act,completions_act,attempts_act,rushing_yards_act,carries_act
0,George Pickens,00-0037247,104.75,11.50,7.25,63.25,0.00,0.0,0.00,0.00,48.0,7,4,0,0,0.0,0
1,David Njoku,00-0033885,85.25,10.25,9.00,31.50,0.00,0.0,0.00,0.00,9.0,5,1,0,0,0.0,0
2,Jerry Jeudy,00-0036407,81.00,8.00,5.00,49.00,0.00,0.0,0.00,0.00,85.0,6,6,0,0,0.0,0
3,Pat Freiermuth,00-0036894,39.50,4.00,3.25,8.00,0.00,0.0,0.00,0.00,59.0,4,4,0,0,0.0,0
4,Cedric Tillman,00-0038979,38.25,7.25,4.00,27.00,0.00,0.0,0.00,0.00,28.0,4,2,0,0,0.0,0
5,Jaylen Warren,00-0037228,29.00,3.00,3.00,10.00,0.00,0.0,60.00,15.00,19.0,5,3,0,0,45.0,11
6,Jerome Ford,00-0037267,27.00,2.25,2.25,4.00,0.00,0.0,22.25,6.25,8.0,1,1,0,0,19.0,4
7,Najee Harris,00-0036893,26.25,3.00,2.00,2.00,0.00,0.0,102.25,23.00,13.0,2,2,0,0,41.0,16
8,Elijah Moore,00-0036980,23.75,4.25,2.25,17.25,0.00,0.0,0.00,0.00,21.0,5,3,0,0,0.0,0
9,Darnell Washington,00-0038558,20.25,4.00,3.00,14.00,0.00,0.0,0.00,0.00,14.0,3,3,0,0,0.0,0


In [ ]:
comb[['air_yards',
 'attempts_act',
 'attempts_pred',
 'carries_act',
 'carries_pred',
 'completions_act',
 'completions_pred',
 'id',
 'name',
 'receiving_yards_act',
 'receiving_yards_pred',
 'receptions_act',
 'receptions_pred',
 'rushing_yards_act',
 'rushing_yards_pred',
 'targets_act',
 'targets_pred']]

,air_yards,attempts_act,attempts_pred,carries_act,carries_pred,completions_act,completions_pred,id,name,receiving_yards_act,receiving_yards_pred,receptions_act,receptions_pred,rushing_yards_act,rushing_yards_pred,targets_act,targets_pred
0,63.25,0,0.0,0,0.00,0,0.00,00-0037247,George Pickens,48.0,104.75,4,7.25,0.0,0.00,7,11.50
1,31.50,0,0.0,0,0.00,0,0.00,00-0033885,David Njoku,9.0,85.25,1,9.00,0.0,0.00,5,10.25
2,49.00,0,0.0,0,0.00,0,0.00,00-0036407,Jerry Jeudy,85.0,81.00,6,5.00,0.0,0.00,6,8.00
3,8.00,0,0.0,0,0.00,0,0.00,00-0036894,Pat Freiermuth,59.0,39.50,4,3.25,0.0,0.00,4,4.00
4,27.00,0,0.0,0,0.00,0,0.00,00-0038979,Cedric Tillman,28.0,38.25,2,4.00,0.0,0.00,4,7.25
5,10.00,0,0.0,11,15.00,0,0.00,00-0037228,Jaylen Warren,19.0,29.00,3,3.00,45.0,60.00,5,3.00
6,4.00,0,0.0,4,6.25,0,0.00,00-0037267,Jerome Ford,8.0,27.00,1,2.25,19.0,22.25,1,2.25
7,2.00,0,0.0,16,23.00,0,0.00,00-0036893,Najee Harris,13.0,26.25,2,2.00,41.0,102.25,2,3.00
8,17.25,0,0.0,0,0.00,0,0.00,00-0036980,Elijah Moore,21.0,23.75,3,2.25,0.0,0.00,5,4.25
9,14.00,0,0.0,0,0.00,0,0.00,00-0038558,Darnell Washington,14.0,20.25,3,3.00,0.0,0.00,3,4.00
